# Imodulon Radiate Analysis
#### Select imodulon from the given list

In [1]:
imodulons = ['ArgR', 'Histidine', 'Isoleucine', 'Leucine', 'MetJ', 'Tryptophan', 'Tyr']
selected_imodulon = 'Tryptophan'

### Definition:
Given list of nodes (source nodes, e.g. genes, metabolites), find what nodes are influenced by the source nodes , and what nodes influence the source nodes.

The algorithm we are using is called __PageRank__. 

The __PageRank__ algorithm measures the importance of each node within the graph, based on the number incoming relationships and the importance of the corresponding source nodes.  

__Personalized PageRank__ is a variation of PageRank which is biased towards a set of source nodes.

<img src="img/personalized_pagerank.png" width="400" height="200" />

### Steps to run Radiate Analysis
1. Connect to arango database
2. Find input nodes (source nodes) in arango database
3. Load the whole network graph from arango to memory and create a networkx graph. NetworkX is a python network library.
4. Perform radiate analysis
    - Run personalized pagerank algorithm using source nodes to get pagerank values for each nodes that the source nodes can reach (forward direction). Those are the nodes influenced by the source nodes
    - Run personalized reverse pagerank using source nodes to get the reverce pagerank values for each nodes that can reach the source nodes
    - Export the pagerank and reverse pagerank values into __excel file__
5. The user analyzes the pagerank values (sorting, filtering etc), and select the rows that are interesting
6. Create radiate traces for selected nodes

In [2]:
# install lifelike_gds package if not already installed (e.g. running in Google Colab)
import importlib

if importlib.util.find_spec('lifelike_gds') is None:
  !pip install git+https://github.com/SBRG/GDS-Public

# provide the path to the notebook folder in the github repository in case the notebook is run in Google Colab
github_path = 'SBRG/GDS-Public/main/notebooks/UCSD_Workshop/iModulon'

In [3]:
import os
import pandas as pd
import warnings
from pathlib import PurePosixPath

In [4]:
from lifelike_gds.arango_network.biocyc import *
from lifelike_gds.arango_network.radiate_trace import RadiateTrace

/Users/dommas/.local/share/virtualenvs/GDS-Public-uKBS7cUg/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
warnings.filterwarnings('ignore')

In [6]:
input_dir = PurePosixPath('input')
output_dir = PurePosixPath('output')
os.makedirs(output_dir, 0o777, True)
# gds database name
db_name = 'ecocyc-25'
# gds database version, free text, that can be used to describe the graph
db_version = 'ecocyc 25.5'

## 1. Connect to arango database.
If use BioCyc databases (e.g. EcoCyc, HumanCyc), use Class BioCycDB.  
If use Reactome database, use Class ReactomeDB. 

In [7]:
# set database uri, username and password. 
# dbname is the arango database name for the running arango instance. The default database name is 'arango'
dbname = os.getenv('ARANGO_DATABASE', db_name)

database = BiocycDB(dbname)

## 2. Find input nodes (source nodes) in arango database

#### Read selected imodulon genes, and find the gene nodes in arango database.

In [ ]:
# get source genes 
source_file_path = input_dir / f'{selected_imodulon}_gene_table.csv'
if os.path.isfile(source_file_path):
  source_file_ref = source_file_path
else:
  # if does not exist localy, pull from github
  source_file_ref = f'https://raw.githubusercontent.com/{github_path}/{source_file_path}'

In [8]:
df = pd.read_csv(source_file_ref)
print(len(df))
df.head()

9


,locus,gene_weight,gene_name,gene_product,cog,operon,regulator,L-tryptophan,Trp-tRNA,TrpR,link
0,b1264,0.362185,trpE,anthranilate synthase subunit TrpE,Amino acid transport and metabolism,trpLEDCBA,"RpoD,TrpR,Trp-tRNA,rydC",False,True,True,https://ecocyc.org/gene?orgid=ECOLI&id=EG11028
1,b1263,0.358054,trpD,anthranilate synthase subunit TrpD,Nucleotide transport and metabolism,trpLEDCBA,"RpoD,TrpR,Trp-tRNA",False,True,True,https://ecocyc.org/gene?orgid=ECOLI&id=EG11027
2,b3161,0.277386,mtr,tryptophan:H(+) symporter Mtr,Amino acid transport and metabolism,mtr,"RpoD,IHF,HU,TyrR,TrpR,Lrp",False,False,True,https://ecocyc.org/gene?orgid=ECOLI&id=EG10617
3,b1262,0.267177,trpC,fused indole-3-glycerol phosphate synthase/pho...,Amino acid transport and metabolism,trpLEDCBA,"RpoD,ppGpp,TrpR,Trp-tRNA",False,True,True,https://ecocyc.org/gene?orgid=ECOLI&id=EG11026
4,b1261,0.259091,trpB,tryptophan synthase subunit beta,Amino acid transport and metabolism,trpLEDCBA,"RpoD,TrpR,Trp-tRNA",False,True,True,https://ecocyc.org/gene?orgid=ECOLI&id=EG11025


In [9]:
source_genes = [n for n in df['gene_name']]
source_nodes = database.get_nodes_by_attr(source_genes, 'name', 'Gene')
print(f"source genes: {len(source_genes)}, nodes: {len(source_nodes)}")

source genes: 9, nodes: 13


## 3. Load the whole network graph from arango to memory and create a networkx graph

Create a RadiateTrace instance.  
RadiateTrace is a subclass of TraceGraphNx.  TraceGraphNx has a property __graph__, that is a networkx graph. After the graph is created by using data from arango graph database, all the algorithms and traces can be run using the python networkx library.

In [10]:
tracegraph = RadiateTrace(Biocyc(database))
# set up output directory where the excel and graph files will write to
tracegraph.datadir = output_dir
# initiate tracegraph by loading graph data from arango
tracegraph.init_default_graph()

INFO: MultiDirectedGraph with 33428 nodes and 37886 edges


## 4. Perform Radiate Analysis
Run personalized pagerank analysis and export values to excel file.

Pagerank analysis is performed using networkx graph that contains a set of nodes and set of edges. 

#### Set node set for source nodes
A node set is a list of node ids with a set name and description.

In [11]:
# node set name
SOURCE_SET = f'{selected_imodulon}_genes'
# node set description
source_desc = f'{selected_imodulon} genes'
# add the node set to graph
tracegraph.set_node_set_from_arango_nodes(source_nodes, name=SOURCE_SET, desc=source_desc)

#### Call export_pagerank_data
The method export_pagerank_data in RadiateTrace contains a few steps to generate the excel file
parameters: 
- sources:  The node set name for the source nodes
- direction: default is __both__. If forward, run pagerank; if reverse, run reverse pagerank; if both, run both forward and reverse pagerank. 
- num_nodes: the nodes of top pagerank or reverse pagerank nodes that will write to the excel file.  The default is 2000. 

In the exported excel file, there is also a column named nReach (or rev_nReach), indicating how many source nodes can be reached by the node in the row.  

The method will write an excel file with two tabs, one for pageranks and one for reverse pageranks.

In [12]:
# get a copy of the original graph, including source node set
tracegraph.graph = tracegraph.orig_graph.copy()

outfile_name = f"Radiate_analysis_for_{SOURCE_SET}.xlsx"
tracegraph.export_pagerank_data(SOURCE_SET, outfile_name, direction='both', num_nodes=4000)

INFO: set pagerank and num reach for Tryptophan_genes
INFO: export top 4000 pagerank data into output/Radiate_analysis_for_Tryptophan_genes.xlsx


## 5. Analyze the pagerank output file (excel), and select interesting rows for further analysis

Suggestion:   
Add a column 'select' for selecting top pagerank nodes, and set any selected rows to 1   

## 6. Create radiate traces for the selected nodes

#### Read manually selected top ranked nodes from the previous generated pagerank excel file
We will read the columns 'select' to get the selected rows. The excel file has two tabs. We will read the selected nodes for pageranks from the 'pageranks' tab, and selected nodes for reverse pageranks from the 'reverse pageranks' tab.

In [13]:
rankfile_path = input_dir / f'Radiate_analysis_for_{SOURCE_SET}_select.xlsx'
if os.path.isfile(rankfile_path):
  rankfile_ref = rankfile_path
else:
  # if does not exist localy, pull from github
  rankfile_ref = f'https://raw.githubusercontent.com/{github_path}/{rankfile_path}'

In [14]:
df_pagerank = pd.read_excel(rankfile_ref, sheet_name='pageranks', usecols=['eid', 'select'])
df_rev_pagerank = pd.read_excel(rankfile_ref, sheet_name='reverse pageranks', usecols=['eid', 'select'])

#### Get selected nodes for forward traces

In [15]:
df_select = df_pagerank[df_pagerank['select']==1]
selected = [id for id in df_select['eid']]
selected_nodes = database.get_nodes_by_attr(selected, 'eid')
print(f"selected: {selected}. length {len(selected_nodes)}")

selected: ['COMPLETE-ARO-PWY', 'TRP', 'TRPSYN-PWY', 'ALL-CHORISMATE-PWY', 'TRANS-RXN-76', 'ANTHRANSYN-CPLX', 'ANTHRANSYN-RXN', 'INDOLE']. length 8


#### Get selected nodes for reverse traces

In [16]:
df_rev_select = df_rev_pagerank[df_rev_pagerank['select']==1]
rev_selected = [id for id in df_rev_select['eid']]
rev_selected_nodes = database.get_nodes_by_attr(rev_selected, 'eid')
print('rev_selected', rev_selected, 'len:', len(rev_selected))

rev_selected ['TU00067', 'CPLX-125', 'TU3', 'PM00368', 'PM00094', 'TERM0-1154', 'TRP'] len: 7


#### Get the original trace graph

In [17]:
# get a copy of the original graph, including source node set
tracegraph.graph = tracegraph.orig_graph.copy()

#### Set node sets for selected nodes

In [18]:
# set selected node set
SELECTED_SET = 'top_pagerank_nodes'
tracegraph.set_node_set_from_arango_nodes(selected_nodes, SELECTED_SET, 'selected top pagerank nodes')

# set rev_selected node set
REV_SELECTED_SET = 'top_rev_pagerank_nodes'
tracegraph.set_node_set_from_arango_nodes(rev_selected_nodes, REV_SELECTED_SET, 'selected top rev pagerank nodes')

#### Add traces and write to graph file for visualization
Add radiate traces using selected nodes

In [ ]:
# set pagerank and reverse pagerank
pagerank_prop = 'pagerank'
tracegraph.set_pagerank(SOURCE_SET, pagerank_prop)
rev_pagerank_prop = 'rev_pagerank'
tracegraph.set_pagerank(SOURCE_SET, rev_pagerank_prop, reverse=True)

# add graph description
tracegraph.add_graph_description(f'Database: {db_version}\n')

# add traces from source genes to each selected nodes
tracegraph.add_traces_from_sources_to_each_selected_nodes(selected_nodes, SOURCE_SET, weighted_prop=pagerank_prop)

# add traces from source genes to all selected nodes
tracegraph.add_trace_from_sources_to_all_selected_nodes(SELECTED_SET, SOURCE_SET, weighted_prop=pagerank_prop)

# add traces from each selected nodes to SOURCE_SET genes
tracegraph.add_traces_from_each_selected_nodes_to_targets(rev_selected_nodes,
                                                           SOURCE_SET, weighted_prop=rev_pagerank_prop)

# add traces from all reverse-selected nodes to SOURCE_SET
tracegraph.add_trace_from_all_selected_nodes_to_targets(REV_SELECTED_SET, SOURCE_SET, weighted_prop=rev_pagerank_prop)

# write all traces into one graph file
graph_file = f'Radiate_traces_for_{SOURCE_SET}.graph'
tracegraph.write_to_sankey_file(graph_file)

INFO: Adding trace network Tryptophan_genes to superpathway of chorismate metabolism #1
INFO: Adding trace network Tryptophan_genes to ANTHRANSYN-CPLX #2
INFO: Adding trace network Tryptophan_genes to Anthranilate synthase-RXN #3
INFO: Adding trace network Tryptophan_genes to superpathway of aromatic amino acid biosynthesis #4
INFO: Adding trace network Tryptophan_genes to indole #5
INFO: Adding trace network Tryptophan_genes to tryptophan:proton symport-RXN #6
INFO: Adding trace network Tryptophan_genes to L-tryptophan #7
INFO: Adding trace network Tryptophan_genes to L-tryptophan biosynthesis #8
INFO: Adding trace network from Tryptophan_genes to top_pagerank_nodes
INFO: Adding trace network from TrpR-Tryptophan DNA-binding transcriptional repressor #1 to Tryptophan_genes
ERROR: Target 18262 cannot be reachedfrom given sources
ERROR: Target 13599 cannot be reachedfrom given sources
ERROR: Target 18262 cannot be reachedfrom given sources
ERROR: Target 13599 cannot be reachedfrom given